# Neural Networks

neural netowrks can be constructed using the `torch.nn` package. the `nn.Module` contains layers and a method called `forward(input)` that passes it forward and returns the output. 

A simple neural network is trained as follows: 

- define the neural network with the learnable parameters/weights
- iterate over the dataset of inputs
- forward pass through the network
- compute the loss (how far it is from being correct with the loss, even loss function)
- propagate the gradients back into the network's parameters
- update the weights of the network, usikng a simple rule like `w = w - lr * grad`

# Define a simple network using this cnn style of networks:

![mnist classification cnn](figures/mnist.png)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    '''
    Note that this net is not exactly the same as the above image
    '''

    def __init__(self, *args, **kwargs):
        super(Net, self).__init__(*args, **kwargs)
        # first, take 1 input channel, 6 output channel, 5x5 square convolution
        # note that these convolution blocks are image size agnostic. we just have to keep the image size in mind when we're desigining this netowkr
        # for example, the mnist images are 32x32 in size. 
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        # then use linear operations
        self.fc1 = nn.Linear(in_features=16*5*5, out_features=128) # note that the 5*5 is from the image dimension but I'm not sure about this yet
        self.fc2 = nn.Linear(in_features=128, out_features=64)
        self.fc3 = nn.Linear(in_features=64, out_features=10)

    def forward(self, input):
        # the first convolution layer C1, in inpout image, 6 output channnels
        # outputs tensor with size (N, 6, 28, 28), N is batch size, reduces image 32 - 5 (kernel size) +1= 28
        c1 = F.relu(input=self.conv1(input))
        # subsample this layer with pooling
        # this pools the information from the 28x28 layer with a square that is 2x2 and outputs the max of those values, which is 14x14
        s2 = F.max_pool2d(input=c1, kernel_size=(2, 2))
        # convolution layer agian, l3, 6 channels in and 16 out
        # out dimensions is 14 - 5 + 1 = 10
        c3 = F.relu(input=self.conv2(s2))
        # another pooling layer, reduces the 14x14 layer to a 7x7 layer
        s4 = F.max_pool2d(input=c3, kernel_size=(2, 2))
        # flatten this output to prep for the linear layers:not sure which flatten dimension to use
        s4 = torch.flatten(input=s4) # out goes a 16*5*5 = 400 dimensional nerual network
        # now for the activation functions for the linear layers, this is fairly straightforward
        f5 = F.relu(input=self.fc1(s4)) # out goes a N, 128 dimensional output
        f6 = F.relu(input=self.fc2(f5)) # out N, 64
        output = self.fc3(f6)
        return output
    
net = Net()
print(net)


Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)


note that while the `forward` function was just defined, the `backward` function where the gradients are computed is automatically defined with `autograd` whoooo. use any Tensor operation you want in the `forward` function

you can also get the number of learnable parameters in `net.parameters()`

In [4]:
net.parameters() #should return a generator object

<generator object Module.parameters at 0x78d0b423bae0>

In [7]:
params = list(net.parameters())
len(params) # 10 layers?

10

In [13]:
# for instance, this is the randomly selected 5x5 convolution window for the first layer.
params[0][0]

tensor([[[ 0.1044, -0.1355,  0.1908,  0.0707, -0.1965],
         [ 0.0639,  0.1531, -0.0630,  0.0156, -0.1934],
         [ 0.0881, -0.0381, -0.0146, -0.0249,  0.0553],
         [-0.0273,  0.1094, -0.1052,  0.1564,  0.0914],
         [ 0.0116, -0.0097, -0.1954, -0.0093,  0.1783]]],
       grad_fn=<SelectBackward0>)

In [ ]:
# this is the one for the 3rd layer, 5th block along the length, and 2nd 5x5 conv block

params[2][4][1]

tensor([[ 0.0128, -0.0482, -0.0798,  0.0624, -0.0147],
        [-0.0268, -0.0454,  0.0803, -0.0301,  0.0623],
        [ 0.0436, -0.0441, -0.0191, -0.0524,  0.0160],
        [ 0.0390,  0.0655, -0.0467, -0.0086,  0.0659],
        [-0.0019, -0.0101,  0.0623,  0.0391, -0.0108]],
       grad_fn=<SelectBackward0>)

In [ ]:
for layer, parameter in enumerate(params):
    print(f"layer {layer}")
    print(parameter[0].size()) 

layer 0
torch.Size([1, 5, 5])
layer 1
torch.Size([])
layer 2
torch.Size([6, 5, 5])
layer 3
torch.Size([])
layer 4
torch.Size([400])
layer 5
torch.Size([])
layer 6
torch.Size([128])
layer 7
torch.Size([])
layer 8
torch.Size([64])
layer 9
torch.Size([])


note that there are a few that do not have any paramters. these are, you guessed it, the functional parts. they are functions, and don't have paramters (usually)

In [33]:
# random input

input = torch.randn(size=(1, 1, 32, 32))
input

tensor([[[[ 6.7301e-01, -5.7621e-01,  2.4888e-01,  ...,  6.2546e-01,
           -3.9181e-01,  2.2146e+00],
          [ 4.2308e-01, -4.5661e-01, -2.7623e-01,  ..., -1.8511e+00,
           -1.0529e+00,  5.5897e-01],
          [ 7.5715e-01, -1.2491e-04, -4.6608e-01,  ...,  5.8265e-01,
            5.6940e-01, -1.3647e+00],
          ...,
          [-9.1382e-01, -2.9432e-01,  1.6746e+00,  ...,  9.0933e-01,
            1.5600e+00, -1.2230e+00],
          [-6.1336e-01, -2.0824e-01, -9.8287e-01,  ..., -1.7465e-01,
           -7.1466e-01,  3.6712e-01],
          [-2.8719e-01,  3.0115e-02,  6.9557e-01,  ..., -3.2516e-01,
           -2.6736e-01, -2.4108e+00]]]])

In [34]:
out = net(input)
out

tensor([ 0.0319,  0.0830,  0.1436, -0.1129,  0.0279,  0.0280,  0.1433, -0.0944,
        -0.0667,  0.0990], grad_fn=<ViewBackward0>)

what pytorch will do is accumulate the gradients for all the backward passes. this means that for each batch, the new gradients will be added ot the existing ones, and you have to set them to zero if you want to do so before each new training iteration/batch. training will break if the gradients are not cleared. for each batch, you want to update the gradient accordingly. sometimes, you want to accumulate the gradients, say, to save memory and simulate a larger batch size for a smaller GPU

In [35]:
net.zero_grad()

In [ ]:
out.backward(torch.flatten(torch.randn(1, 10))) # with a random label

tensor([ 0.0319,  0.0830,  0.1436, -0.1129,  0.0279,  0.0280,  0.1433, -0.0944,
        -0.0667,  0.0990], grad_fn=<ViewBackward0>)

> Note that `torch.nn` ony suppoprts batches and **not single samples**. this means that the if you have a single sample, simply use `input.unsqueeze(0)`

now just to recap

- `torch.Tensor`: a multidimensional array with support for `autograd` and it's operations, holds gradients with respect to the tensor itself
- `nn.Module`, neural netowrk module, convenient way of encapsulating the parameters
    - includes helpers to move them to the GPU, io, etc
- `nn.Parameter` sort of a tensor that automatically registered as a parameter when assigned an attribute to a MOdule
- `autograd.Function` implements the forward/backward passes of a module. every `Tensor` creates at least a single `Function` note that connects to functions that created a tensor and encodes its history.

# Loss functions

a loss function compues a value that is supposed to illustrate the difference (loss) of the io values. a simple loss is Mean Square Error loss, which is `nn.MSELoss`, computing the mse 

In [37]:
torch.randn(10)

tensor([ 0.9250, -2.5162, -0.0027, -1.1703,  1.9402, -0.0717,  0.4340,  0.2116,
         0.2472, -1.8958])

In [ ]:
output = net(input)
target = torch.randn(10)
output, target

(tensor([ 0.0319,  0.0830,  0.1436, -0.1129,  0.0279,  0.0280,  0.1433, -0.0944,
         -0.0667,  0.0990], grad_fn=<ViewBackward0>),
 tensor([-0.1414,  0.4394,  2.1626, -0.3430, -1.3231,  0.6644,  1.6357, -0.8177,
          0.1454, -0.3506]))

In [ ]:
# defining the criterion
criterion = nn.MSELoss()

loss = criterion(output, target) # must be in the order of (prediction, target)
print(loss)

tensor(0.9514, grad_fn=<MseLossBackward0>)


if you want to follow the gradients backwards

In [43]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

# backpropagation

backpropagation, especially for linear layers are very straightforward. 

In [69]:
net.zero_grad()     # this is the gradient zeroed out/buffer cleared for all parameters

print("conv1.bias.grad before the backprop")
net.conv1.bias.grad

conv1.bias.grad before the backprop


In [70]:
loss.backward()     # after the backprop

print("conv1.bias.grad before the backprop")
net.conv1.bias.grad

conv1.bias.grad before the backprop


tensor([-0.0082, -0.0165, -0.0036, -0.0217, -0.0060, -0.0108])

# update the weights of the network

now we just have to update the weights

In [71]:
# this is a simple way to do this using code:
lr = 0.01
for p in net.parameters():
    p.data.sub_(p.grad.data * lr) # with the _ decorator at the end, it just is an inplace operation

in order to fully utililze the power of pytorch, you should really just use an abstracted optimizer for this

In [72]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=lr)

In [73]:
# then in the trainig loop:

optimizer.zero_grad() # zero gradients
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # does the update itself